In [ ]:
import sys
sys.path.append("../Pupillengröße/Skripte/")
#%matplotlib widget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging as log
import checkdata
import importlib
from IPython.display import display
def display_all(df):
    from IPython.display import HTML, display
    IPython.display(IPython.HTML(df.to_html()))


In [ ]:
import preprocessing
import plotting
import math
import os

importlib.reload(preprocessing)
importlib.reload(plotting)
importlib.reload(checkdata)


In [ ]:

import preprocessing
import plotting
import math
import os, getpass
import pup_config

importlib.reload(preprocessing)
importlib.reload(plotting)
importlib.reload(pup_config)
#importlib.reload(checkdata)

def plot(df,text,xcol,ycol):
    plotting.plot_preprocessed(df,text,xcol,ycol)
    
def noplot(df,text,xcol,ycol):
    pass

subject_id="PJ25_1_Ruhe"
data_dir=pup_config.data_dir

field="diameter_3d"
ts="pupil_timestamp"

config=preprocessing.create_process_config(0,field,subject_id,data_dir)

# Set this to False, if you want to process the data
# Set this to True, if you only want to return an array of data checks 

#config.validate_only=True

eye0=preprocessing.process(config,display)
# display(eye0)

if True:
    if config.validate_only:
        # no data, just a structure containing annotation indices and error estimates
        display(eye0) 
    else:
        for what_pf in eye0.frames:
            if what_pf.valid:
                what=what_pf.data
                ts=what['pupil_timestamp'].iloc[0]
                plot(what,f"frame @{ts}","pupil_timestamp_based",f"{field}_original")
                plot(what,f"gated @{ts}","pupil_timestamp_based",f"{field}_gated")
                plot(what,f"recon @{ts}","pupil_timestamp_based",f"{field}_rec")
                plot(what,f"interp @{ts}","pupil_timestamp_based",f"{field}_rec_interp_100")

        if True:
            for what_pf in eye0.frames:
                if what_pf.valid:
                    what=what_pf.data
                plotting.plot_preprocessed(what,f"frame @what.pupil_timestamp[0]","pupil_timestamp_based",f"{field}_gated")





In [ ]:
display(f"Anzahl der Messungen: {len(eye0.frames)}")
sub=None
i=0
for what_pf in eye0.frames:
    #display(what_pf)
    if what_pf.valid:
        what=what_pf.data
        ts=what_pf.annotation_ts
        # ts=what['pupil_timestamp'].iloc[0]
        # preprocessing.create_baseline_column(what, f'{field}_original',f'{field}_original_baseline')
        sub=what.plot(ax=sub, x='pupil_timestamp_based', y=f'{field}_baseline', ylabel='baseline adjusted pupil size', kind='line',label=ts)
        sub.set_ylabel(f'measurement {subject_id}, {field}')
         # Adjust the position of the legend
        legend = sub.legend()
        legend.set_bbox_to_anchor((1.5, 1))  # Adjust the position of the legend box

In [ ]:
#display(eye0.num_valid, len(eye0.frames))
display([(f.index,f.baseline_mean,f.annotation_ts,f.zscore,f.stage,f.remark) for f in eye0.frames])


zscores=[f.zscore for f in eye0.frames if f.valid or f.stage=="zscore"]
plt.hist(zscores)

In [ ]:
importlib.reload(preprocessing)
av_df=preprocessing.average_frames_by_resample(eye0,f'{field}_baseline',interval='100ms') 
av_df.plot(x='ts',y=f'{field}_baseline')
av_df=preprocessing.average_frames_by_binning(eye0,f'{field}_baseline',interval_ms=100)
av_df.plot(x='ts',y=f'{field}_baseline')
#av_df=preprocessing.average_frames3(eye0,f'{field}_baseline',interval_ms=10)
#av_df.plot(x='ts',y=f'{field}_baseline')
display(eye0.config)
display(av_df)
